In [20]:
import requests
import json
from IPython.display import JSON
from keys import *

In [21]:
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/YMML/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': "77cd5e6190mshbabec6d3965dd94p1667efjsn21302f39a238"
    }

response = requests.request("GET", url, headers=headers, params=querystring)


In [23]:
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [24]:
flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']


In [26]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
    })
    return(flights_df)

flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()

,flight_number,from,airline,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,VA 204,Adelaide,Virgin Australia,2025-08-31 21:55Z,2025-09-01 07:55+10:00
0,VA 726,Gold,Virgin Australia,2025-08-31 22:25Z,2025-09-01 08:25+10:00
0,TG 4844,Wellington,Thai International,2025-08-31 22:10Z,2025-09-01 08:10+10:00
0,VA 7153,Wellington,Virgin Australia,2025-08-31 22:10Z,2025-09-01 08:10+10:00
0,SQ 4330,Wellington,Singapore,2025-08-31 22:10Z,2025-09-01 08:10+10:00
